In [219]:
!pip install pyspark
!pip install kaggle
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [220]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as func
from pyspark.sql.types import StringType,FloatType
import nltk

In [221]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('advertising (1).csv').getOrCreate()
spark

In [222]:
df = spark.read.csv("advertising (1).csv", header=True) #Loading the data

In [223]:
df.show()

+------------------------+---+-----------+--------------------+--------------------+-----------------+------+--------------------+----------------+-------------+
|Daily Time Spent on Site|Age|Area Income|Daily Internet Usage|       Ad Topic Line|             City|  Male|             Country|       Timestamp|Clicked on Ad|
+------------------------+---+-----------+--------------------+--------------------+-----------------+------+--------------------+----------------+-------------+
|                   68.95| 35|    61833.9|              256.09|Cloned 5thgenerat...|      Wrightburgh|female|             Tunisia|27/03/2016 00:53|            0|
|                   80.23| 31|   68441.85|              193.77|Monitored nationa...|        West Jodi|  male|               Nauru|04/04/2016 01:39|            0|
|                   69.47| 26|   59785.94|               236.5|Organic bottom-li...|         Davidton|female|          San Marino|13/03/2016 20:35|            0|
|                   74.15| 2

In [224]:
#df = df.withColumnRenamed('_c0','target').withColumnRenamed('_c1','id').withColumnRenamed('_c2','date')\.withColumnRenamed('_c3','flag').withColumnRenamed('_c4','user').withColumnRenamed('_c5','text')
df.show() #renaming column

+------------------------+---+-----------+--------------------+--------------------+-----------------+------+--------------------+----------------+-------------+
|Daily Time Spent on Site|Age|Area Income|Daily Internet Usage|       Ad Topic Line|             City|  Male|             Country|       Timestamp|Clicked on Ad|
+------------------------+---+-----------+--------------------+--------------------+-----------------+------+--------------------+----------------+-------------+
|                   68.95| 35|    61833.9|              256.09|Cloned 5thgenerat...|      Wrightburgh|female|             Tunisia|27/03/2016 00:53|            0|
|                   80.23| 31|   68441.85|              193.77|Monitored nationa...|        West Jodi|  male|               Nauru|04/04/2016 01:39|            0|
|                   69.47| 26|   59785.94|               236.5|Organic bottom-li...|         Davidton|female|          San Marino|13/03/2016 20:35|            0|
|                   74.15| 2

In [225]:
df.printSchema() 

root
 |-- Daily Time Spent on Site: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Area Income: string (nullable = true)
 |-- Daily Internet Usage: string (nullable = true)
 |-- Ad Topic Line: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Male: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Clicked on Ad: string (nullable = true)



In [226]:
df.head(5)

[Row(Daily Time Spent on Site='68.95', Age='35', Area Income='61833.9', Daily Internet Usage='256.09', Ad Topic Line='Cloned 5thgeneration orchestration', City='Wrightburgh', Male='female', Country='Tunisia', Timestamp='27/03/2016 00:53', Clicked on Ad='0'),
 Row(Daily Time Spent on Site='80.23', Age='31', Area Income='68441.85', Daily Internet Usage='193.77', Ad Topic Line='Monitored national standardization', City='West Jodi', Male='male', Country='Nauru', Timestamp='04/04/2016 01:39', Clicked on Ad='0'),
 Row(Daily Time Spent on Site='69.47', Age='26', Area Income='59785.94', Daily Internet Usage='236.5', Ad Topic Line='Organic bottom-line service-desk', City='Davidton', Male='female', Country='San Marino', Timestamp='13/03/2016 20:35', Clicked on Ad='0'),
 Row(Daily Time Spent on Site='74.15', Age='29', Area Income='54806.18', Daily Internet Usage='245.89', Ad Topic Line='Triple-buffered reciprocal time-frame', City='West Terrifurt', Male='male', Country='Italy', Timestamp='10/01/2

In [227]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, when, row_number, to_date, year, month, dayofweek
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

In [228]:
# Filter Data
df.filter(df["Area Income"] > 70).show(10)

+------------------------+---+-----------+--------------------+--------------------+----------------+------+----------+----------------+-------------+
|Daily Time Spent on Site|Age|Area Income|Daily Internet Usage|       Ad Topic Line|            City|  Male|   Country|       Timestamp|Clicked on Ad|
+------------------------+---+-----------+--------------------+--------------------+----------------+------+----------+----------------+-------------+
|                   68.95| 35|    61833.9|              256.09|Cloned 5thgenerat...|     Wrightburgh|female|   Tunisia|27/03/2016 00:53|            0|
|                   80.23| 31|   68441.85|              193.77|Monitored nationa...|       West Jodi|  male|     Nauru|04/04/2016 01:39|            0|
|                   69.47| 26|   59785.94|               236.5|Organic bottom-li...|        Davidton|female|San Marino|13/03/2016 20:35|            0|
|                   74.15| 29|   54806.18|              245.89|Triple-buffered r...|  West Ter

In [229]:
# Grouping and Aggregating Data
df.groupBy("Male").agg({"Daily Internet Usage": "sum"}).show(5)

+------+-------------------------+
|  Male|sum(Daily Internet Usage)|
+------+-------------------------+
|female|        92805.90000000005|
|  male|        87194.19999999991|
+------+-------------------------+



In [230]:
# Grouping and Aggregating Data
df.groupBy("City").agg({"Area Income": "mean"}).show(5)

+----------------+----------------+
|            City|avg(Area Income)|
+----------------+----------------+
|      Curtisview|        61275.18|
|      Karenmouth|         69805.7|
|South Davidmouth|        31998.72|
|  Villanuevastad|        65756.36|
|     Tinachester|        69112.84|
+----------------+----------------+
only showing top 5 rows



In [231]:
df.groupBy("Country").agg({"Daily Internet Usage":"sum"}).show(5)

+--------------------+-------------------------+
|             Country|sum(Daily Internet Usage)|
+--------------------+-------------------------+
|                Chad|                   787.52|
|            Paraguay|                    528.4|
|            Anguilla|                  1164.11|
|               Macao|                   377.13|
|Heard Island and ...|                   524.69|
+--------------------+-------------------------+
only showing top 5 rows



In [232]:
# Handling Missing Values
df.fillna(0, subset=["Daily Internet Usage"])

DataFrame[Daily Time Spent on Site: string, Age: string, Area Income: string, Daily Internet Usage: string, Ad Topic Line: string, City: string, Male: string, Country: string, Timestamp: string, Clicked on Ad: string]

In [233]:
# Sorting Data
df.orderBy("City").show(5)

+------------------------+---+-----------+--------------------+--------------------+-------------+------+-------------+----------------+-------------+
|Daily Time Spent on Site|Age|Area Income|Daily Internet Usage|       Ad Topic Line|         City|  Male|      Country|       Timestamp|Clicked on Ad|
+------------------------+---+-----------+--------------------+--------------------+-------------+------+-------------+----------------+-------------+
|                   40.19| 37|   55358.88|              136.99|Right-sized syste...|    Adamsbury|female|       France|18/01/2016 02:51|            1|
|                   45.82| 27|    64410.8|              171.24|Expanded full-ran...|     Adamside|  male|      Eritrea|10/02/2016 20:43|            1|
|                   81.05| 34|   54045.39|               245.5|Configurable asyn...|    Adamsstad|female|     Malaysia|18/05/2016 00:07|            0|
|                   83.26| 40|    70225.6|              187.76|Decentralized for...|     Alanv

In [234]:
# Renaming Columns
df = df.withColumnRenamed("Male", "Gender").show(5)

+------------------------+---+-----------+--------------------+--------------------+--------------+------+----------+----------------+-------------+
|Daily Time Spent on Site|Age|Area Income|Daily Internet Usage|       Ad Topic Line|          City|Gender|   Country|       Timestamp|Clicked on Ad|
+------------------------+---+-----------+--------------------+--------------------+--------------+------+----------+----------------+-------------+
|                   68.95| 35|    61833.9|              256.09|Cloned 5thgenerat...|   Wrightburgh|female|   Tunisia|27/03/2016 00:53|            0|
|                   80.23| 31|   68441.85|              193.77|Monitored nationa...|     West Jodi|  male|     Nauru|04/04/2016 01:39|            0|
|                   69.47| 26|   59785.94|               236.5|Organic bottom-li...|      Davidton|female|San Marino|13/03/2016 20:35|            0|
|                   74.15| 29|   54806.18|              245.89|Triple-buffered r...|West Terrifurt|  male|

In [235]:
#from pyspark.sql.functions import concat_ws

In [236]:
# Concatenating Columns
#df = df.withColumn("NewColumn", col("City") * 2)
df = df.withColumn("concatenated_column", concat(df["City"], df["Country"])).show(5)

AttributeError: 'NoneType' object has no attribute 'withColumn'

In [194]:
# Dropping Columns
df = df.drop("concatenated_column")

AttributeError: 'NoneType' object has no attribute 'drop'

In [195]:
# Distinct Values in a Column
df.select("Area Income").distinct().show()

AttributeError: 'NoneType' object has no attribute 'select'